# Podemos usar ObjectScript desde Jupyter!

In [1]:
Set hello = "helloworld2"
zw hello

hello="helloworld2"


# Usando Python desde ObjectSCript

In [2]:
set datetime = ##class(%SYS.Python).Import("datetime")
zw datetime
zw datetime.date.today().isoformat()

datetime=3@%SYS.Python  ; <module 'datetime' from '/usr/lib/python3.8/datetime.py'>  ; <OREF>
"2022-03-29"


In [3]:
set rslt = ##class(%SYS.Python).Run("print('hello world')")

hello world


In [4]:
zn "USER"
do ##class(%EnsembleMgr).EnableNamespace("USER") 

## Generación de PDFs usando la librería reportlab

In [5]:
do ##class(Demo.reportlab).CreateSamplePDF("/opt/irisapp/misc/sample.pdf")

## Georeferencias: geopy

In [6]:
set geopy = ##class(%SYS.Python).Import("geopy")

In [7]:
set args = { "user_agent": "Embedded Python" }

In [8]:
set geolocator = geopy.Nominatim(args...)

In [9]:
set flatiron = geolocator.geocode("175 5th Avenue NYC")

In [10]:
write flatiron.address

Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan Community Board 5, Manhattan, New York, 10010, United States

In [11]:
set cityhall = geolocator.reverse("42.3604099,-71.060181")

In [12]:
write cityhall.address

Government Center, Cambridge Street, Downtown Crossing, Downtown Boston, Boston, Suffolk County, Massachusetts, 02203, United States

In [13]:
set edificio = geolocator.reverse("41.4060085,2.1727896")

In [14]:
zw edificio

edificio=6@%SYS.Python  ; 304, Carrer de la Marina, la Sagrada Família, Eixample, Barcelona, Barcelonès, Barcelona, Catalunya, 08025, España  ; <OREF>


## Paso de datos entre Python y ObjectScript

In [15]:
set builtins = ##class(%SYS.Python).Builtins()

### Python list!

In [16]:
set newport = builtins.list()

In [17]:
do newport.append(41.49008)

In [18]:
do newport.append(-71.312796)
zw newport

newport=17@%SYS.Python  ; [41.49008, -71.312796]  ; <OREF>


In [19]:
set cleveland = builtins.list()

In [20]:
do cleveland.append(41.499498)

In [21]:
do cleveland.append(-81.695391)
zw cleveland

cleveland=36@%SYS.Python  ; [41.499498, -81.695391]  ; <OREF>


In [22]:
set distance = $system.Python.Import("geopy.distance")

In [23]:
set route = distance.distance(newport, cleveland)

In [24]:
zw route.miles

538.3904453677203037


# Cómo llamar a un método, escrito en Python, de una clase ObjecScript

```
Class ObjectScript.Sample.Address Extends %Library.Persistent
{

Property AddressLine1 As %String;

Property AddressLine2 As %String;

Property City As %String;

Property State As %String;

Property PostalCode As %String;

Method Normalize(addr As %String) [ Language = python ]
{

        from scourgify import normalize_address_record
        normalized = normalize_address_record(addr)

        self.AddressLine1 = normalized['address_line_1']
        self.AddressLine2 = normalized['address_line_2']
        self.City = normalized['city']
        self.State = normalized['state']
        self.PostalCode = normalized['postal_code']
}
    
}
```

In [25]:
set a = ##class(ObjectScript.Sample.Address).%New()
do a.Normalize("One Memorial Drive, 8th Floor, Cambridge, Massachusetts 02142")
zw a

a=24@ObjectScript.Sample.Address  ; <OREF>
+----------------- general information ---------------
|      oref value: 24
|      class name: ObjectScript.Sample.Address
| reference count: 2
+----------------- attribute values ------------------
|       %Concurrency = 1  <Set>
|       AddressLine1 = "ONE MEMORIAL DR"
|       AddressLine2 = "FL 8TH"
|               City = "CAMBRIDGE"
|         PostalCode = "02142"
|              State = "MA"
+-----------------------------------------------------


## Usando las funciones internas de Python

In [26]:
set builtins = ##class(%SYS.Python).Import("builtins")

In [27]:
do builtins.print("hello world!")

hello world!


# Nombres
_ está permitido!

In [28]:
set mylist = builtins.list()

In [29]:
zwrite mylist."__class__"

22@%SYS.Python  ; <class 'list'>  ; <OREF>


# Argumentos con nombre
En Python se usan argumentos con nombre cuando definimos un método:
```python
def mymethod(foo=1, bar=2, baz="three"):
    print(f"foo={foo}, bar={bar}, baz={baz}")
```

In [30]:
set args={ "bar": 123, "foo": "foo"}

set obj = ##class(%SYS.Python).Import("mymodule")

set args={ "bar": 123, "foo": "foo"}

do obj.mymethod(args...)

foo=foo, bar=123, baz=three

# Paso de argumentos por referencia
Como python no soporta el paso por referencia hay que usar iris.ref()

```
ClassMethod SandwichSwitch(bread As %String, ByRef filling1 As %String, ByRef filling2 As %String)
{
    set bread = "whole wheat"
    set filling1 = "almond butter"
    set filling2 = "cherry preserves"
}
```

In [31]:
set arg1 = "white bread"
set arg2 = "peanut butter"
set arg3 = "grape jelly"
do ##class(ObjectScript.Sample.Sandwich).SandwichSwitch(arg1, .arg2, .arg3)
 
write arg1

white bread

In [32]:
write arg2


almond butter

In [33]:
write arg3

cherry preserves

En python:
```
import iris
arg1 = "white bread"
arg2 = iris.ref("peanut butter")
arg3 = iris.ref("grape jelly")
iris.cls('User.EmbeddedPython').SandwichSwitch(arg1, arg2, arg3)
```

```shell
>>> arg1
'white bread'
>>> arg2.value
'almond butter'
>>> arg3.value
'cherry preserves'
```

# Excepciones

In [34]:
set canvaslib = ##class(%SYS.Python).Import("reportlab.pdfgen.canvas")
set canvas = canvaslib.Canvas("/tmp/test.pdf")

In [35]:
try { do canvas.drawImage("/tmp/bad.png", 150, 600) } catch { zw "Se ha producido una excepción!" }

"Se ha producido una excepción!"


# True, false, None
Hay que usar %SYS.Python

In [36]:
zw ##class(%SYS.Python).True()

27@%SYS.Python  ; True  ; <OREF>


# Diccionarios: clave-valor

En python:
```
>>> mycar = {
...     "make": "Toyota",
...     "model": "RAV4",
...     "color": "blue"
... }
>>> print(mycar)
{'make': 'Toyota', 'model': 'RAV4', 'color': 'blue'}
>>> print(mycar["color"])
blue
```

In [37]:
set mycar = ##class(%SYS.Python).Builtins().dict()

In [38]:
do mycar.setdefault("make", "Toyota")

In [39]:
do mycar.setdefault("model", "RAV4")

In [40]:
do mycar.setdefault("color", "blue")

In [41]:
zw mycar

mycar=25@%SYS.Python  ; {'make': 'Toyota', 'model': 'RAV4', 'color': 'blue'}  ; <OREF>
